In [3]:
import torch
import gym
import numpy as np
from PIL import Image
from agent import Agent
import os
import collections
import cv2


In [4]:
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

False


In [85]:
class DQNKungFuMaster(gym.Wrapper):
    def __init__(self,render_mode='rgb_array',device='cpu'):
        env = gym.make("ALE/KungFuMaster-v5",render_mode=render_mode)

        super(DQNKungFuMaster, self).__init__(env)
       
        self.image_shape = (84,84)        
        self.device=device

    def step(self,action): #take step and get observation
        total_reward=0
        done = False
        buffer=[]
        for i in range(4): #you dont really want to react on every frame, goup frames 4 at a time
            observation,reward,done,trucated,info=self.env.step(action)
            total_reward+=reward

            proccessed_img = Image.fromarray(observation)
            proccessed_img = proccessed_img.resize(self.image_shape)
            proccessed_img = proccessed_img.convert("L")          

            buffer.append(proccessed_img)      
            if done == True:
                break
        
        images=np.concatenate(buffer[:],axis=1)
        buffer.clear()
        self.showimg+=1
        if(self.showimg>10):
            img_to_show=Image.fromarray(images)
            img_to_show.show()
            self.showimg=0

        


        images = np.array(images)        
        images = torch.from_numpy(images)
        images = images.unsqueeze(0)
        images = images.unsqueeze(0)
        images = images/255.0
        images = images.to(self.device)




        
        total_reward = torch.tensor(total_reward).view(1,-1).float()
        total_reward = total_reward.to(self.device)

        done = torch.tensor(done).view(1,-1)
        done = done.to(self.device)

        #max_frame = max_frame.to(self.device)
        return images, total_reward,done,info

    def reset(self):        
        observation , _ = self.env.reset()        
        #observation = self.process_observation(observation)
        proccessed_img = Image.fromarray(observation)
        proccessed_img = proccessed_img.resize(self.image_shape)
        proccessed_img = proccessed_img.convert("L")  
        images=np.concatenate((proccessed_img,proccessed_img,proccessed_img,proccessed_img),axis=1)
        images = np.array(images)        
        images = torch.from_numpy(images)
        images = images.unsqueeze(0)
        images = images.unsqueeze(0)
        images = images/255.0
        images = images.to(self.device)
        
  
        return observation

    

In [7]:
class DQNKungFuMaster(gym.Wrapper):
    def __init__(self,render_mode='rgb_array',device='cpu'):
        env = gym.make("ALE/KungFuMaster-v5",render_mode=render_mode)

        super(DQNKungFuMaster, self).__init__(env)
        self.showimg=0
        self.image_shape = (84,84)        
        self.device=device

    def step(self,action): #take step and get observation
        total_reward=0
        done = False
        image=None
        for i in range(4): #you dont really want to react on every frame, goup frames 4 at a time
            observation,reward,done,trucated,info=self.env.step(action)
            total_reward+=reward

            proccessed_img = Image.fromarray(observation)
            proccessed_img = proccessed_img.resize(self.image_shape)
            proccessed_img = proccessed_img.convert("L")  
            proccessed_img = self.crop_image(proccessed_img,5,12,20)        
            
            if(i==0):
                image=proccessed_img
            else:
                image=np.concatenate((image,proccessed_img),axis=1)    
            if done == True:
                break
        
        frame=image        
        self.showimg+=1
        if(self.showimg>10):
            img_to_show=Image.fromarray(frame)
            img_to_show.show()
            self.showimg=0      


        frame = np.array(frame)        
        frame = torch.from_numpy(frame)
        frame = frame.unsqueeze(0)
        frame = frame.unsqueeze(0)
        frame = frame/255.0
        frame = frame.to(self.device)

        
        total_reward = torch.tensor(total_reward).view(1,-1).float()
        total_reward = total_reward.to(self.device)

        done = torch.tensor(done).view(1,-1)
        done = done.to(self.device)

        #max_frame = max_frame.to(self.device)
        return frame, total_reward,done,info


    def crop_image(self,image,left_px,top_px,bottom_px):
        width,height = image.size
        cropped_image = image.crop((left_px, top_px, width, height - bottom_px))
        width,height = cropped_image.size
        strip_upper_line=41
        strip_width = 15
        extra_cropped_image_up = cropped_image.crop((0, 0, width, height - strip_upper_line))
        extra_cropped_image_down = cropped_image.crop((0, strip_upper_line-strip_width, width, height))
        new_height=extra_cropped_image_up.height+extra_cropped_image_down.height
        extra_cropped_image = Image.new('L', (width, new_height))
        extra_cropped_image.paste(extra_cropped_image_up, (0, 0))
        extra_cropped_image.paste(extra_cropped_image_down, (0, extra_cropped_image_up.height))
        #print(extra_cropped_image.size) (79, 37) size 
        return extra_cropped_image

    def reset(self):        
        observation , _ = self.env.reset()        
        #observation = self.process_observation(observation)
        proccessed_img = Image.fromarray(observation)
        proccessed_img = proccessed_img.resize(self.image_shape)
        proccessed_img = proccessed_img.convert("L")  
        images=np.concatenate((proccessed_img,proccessed_img,proccessed_img,proccessed_img),axis=1)
        images = np.array(images)        
        images = torch.from_numpy(images)
        images = images.unsqueeze(0)
        images = images.unsqueeze(0)
        images = images/255.0
        images = images.to(self.device)
        
  
        return observation

    

In [91]:
class DQNKungFuMaster(gym.Wrapper):
    def __init__(self,render_mode='rgb_array',device='cpu'):
        env = gym.make("ALE/KungFuMaster-v5",render_mode=render_mode)

        super(DQNKungFuMaster, self).__init__(env)   
        self.image_shape = (84,84)        
        self.device=device

    def step(self,action): #take step and get observation
        total_reward=0
        done = False
        buffer=[]
        for i in range(4): #you dont really want to react on every frame, goup frames 4 at a time
            observation,reward,done,trucated,info=self.env.step(action)
            total_reward+=reward

            proccessed_img = Image.fromarray(observation)
            proccessed_img = proccessed_img.resize(self.image_shape)
            proccessed_img = proccessed_img.convert("L")  
            proccessed_img = self.crop_image(proccessed_img,5,12,20)        

            buffer.append(proccessed_img)      
            if done == True:
                break
        
        frame=np.concatenate(buffer[-1:],axis=1)
        buffer.clear()

        frame = np.array(frame)        
        frame = torch.from_numpy(frame)
        frame = frame.unsqueeze(0)
        frame = frame.unsqueeze(0)
        frame = frame/255.0
        frame = frame.to(self.device)

        
        total_reward = torch.tensor(total_reward).view(1,-1).float()
        total_reward = total_reward.to(self.device)

        done = torch.tensor(done).view(1,-1)
        done = done.to(self.device)

        #max_frame = max_frame.to(self.device)
        return frame, total_reward,done,info


    def crop_image(self,image,left_px,top_px,bottom_px):
        width,height = image.size
        cropped_image = image.crop((left_px, top_px, width, height - bottom_px))
        width,height = cropped_image.size
        strip_upper_line=41
        strip_width = 15
        extra_cropped_image_up = cropped_image.crop((0, 0, width, height - strip_upper_line))
        extra_cropped_image_down = cropped_image.crop((0, strip_upper_line-strip_width, width, height))
        new_height=extra_cropped_image_up.height+extra_cropped_image_down.height
        extra_cropped_image = Image.new('L', (width, new_height))
        extra_cropped_image.paste(extra_cropped_image_up, (0, 0))
        extra_cropped_image.paste(extra_cropped_image_down, (0, extra_cropped_image_up.height))
        #print(extra_cropped_image.size) (79, 37) size 
        return extra_cropped_image

    def reset(self):        
        observation , _ = self.env.reset()        
        #observation = self.process_observation(observation)
        proccessed_img = Image.fromarray(observation)
        proccessed_img = proccessed_img.resize(self.image_shape)
        proccessed_img = proccessed_img.convert("L")  
        images=np.concatenate((proccessed_img,proccessed_img,proccessed_img,proccessed_img),axis=1)
        images = np.array(images)        
        images = torch.from_numpy(images)
        images = images.unsqueeze(0)
        images = images.unsqueeze(0)
        images = images/255.0
        images = images.to(self.device)
        
  
        return observation

    

In [9]:
environment = DQNKungFuMaster(device=device)
environment.reset()
for _ in range(0,50):
    environment.step(2)

/Users/christosioannidis/anaconda3/envs/ml/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [9]:
environment = DQNKungFuMaster(device=device , render_mode='human')
environment.reset()
for _ in range(0,50):
    environment.step(2)

/Users/christosioannidis/anaconda3/envs/ml/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


: 